In [84]:
from operator import index
!which python

/Users/apabook/Desktop/ds-deployment/.venv/bin/python


In [85]:
import pandas as pd
import requests
import json
import pprint


# Testing HL Api for PULL transfer

---

## Consts

In [86]:
data_space_provider = "http://127.0.0.1:1234"
data_space_consumer = "http://127.0.0.1:1235"

catalog_id = ""
dataservice_id = ""
agreement_id = ""
agreement_pid = ""

## Setup Catalog, Dataservice and agreements

In [87]:
payload = {
    "foaf:homepage": "My catalog in Dataspace provider",
    "dct:title": "My catalog in Dataspace provider"
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
catalog_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': '982d9012-d904-493d-9efc-42d9de53443d',
 '@type': 'dcat:Catalog',
 'dcat:dataset': [],
 'dcat:keyword': '',
 'dcat:service': [],
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': '982d9012-d904-493d-9efc-42d9de53443d',
 'dct:issued': '2024-11-28T15:02:36.467676',
 'dct:modified': None,
 'dct:title': 'My catalog in Dataspace provider',
 'dspace:extraFields': None,
 'dspace:participantId': None,
 'foaf:homepage': 'My catalog in Dataspace provider',
 'odrl:hasPolicy': None}


In [88]:
payload = {
    "dcat:endpointURL": "http://api-test:3000"
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs/" + catalog_id + "/data-services"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
dataservice_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': 'be8ab8ff-4c30-4014-beb4-aaf0e66c8d9e',
 '@type': 'dcat:DataService',
 'dcat:endpointDescription': '',
 'dcat:endpointURL': 'http://api-test:3000',
 'dcat:keyword': '',
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': 'be8ab8ff-4c30-4014-beb4-aaf0e66c8d9e',
 'dct:issued': '2024-11-28T15:02:36.539920',
 'dct:modified': None,
 'dct:title': None,
 'dspace:extraFields': None,
 'odrl:hasPolicy': None}


In [89]:
payload = {
    "dataServiceId": dataservice_id
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/agreements"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
agreement_id = response_as_json["agreement_id"]
agreement_pid = "urn:uuid:" + agreement_id

pprint.pprint(response.json())

{'agreement_id': '62e13174-f8d7-48ef-81c1-fc870b9e3b54',
 'data_service_id': 'be8ab8ff-4c30-4014-beb4-aaf0e66c8d9e',
 'identity': None,
 'identity_token': None}


## Setup transfer

In [90]:
url = data_space_consumer + "/api/v1/setup-transfer"

payload = ""
headers = {}
response = requests.request("POST", url, headers=headers, data=payload)
callbackAddress = response.json()["callbackAddress"]
callbackId = response.json()["callbackId"]
consumerPid = response.json()["consumerPid"]

pprint.pprint(response.json())

{'callbackAddress': 'http://ds-consumer:1235/0c0e6597-504c-4e33-b0f4-09af4d45fdd9',
 'callbackId': '0c0e6597-504c-4e33-b0f4-09af4d45fdd9',
 'consumerPid': 'urn:uuid:be870ca3-17be-468d-9cf4-5bac748af80d'}


## Request transfer

In [91]:
url = data_space_consumer + "/api/v1/request-transfer"

payload = json.dumps({
  "agreementId": agreement_pid,
  "format": "http+pull",
  "dataAddress": None,
  "callbackAddress": callbackAddress,
  "callbackId": callbackId,
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())


{'consumerPid': 'urn:uuid:be870ca3-17be-468d-9cf4-5bac748af80d',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:be870ca3-17be-468d-9cf4-5bac748af80d',
                     'dspace:providerPid': 'urn:uuid:69bf11a8-96f5-4539-9df7-fc11ab0f8612',
                     'dspace:state': 'dspace:REQUESTED'}}


## Get data address

In [92]:
import requests

consumerPidUuid = consumerPid.replace("urn:uuid:", "")
url = data_space_consumer + "/api/v1/data-address/" + consumerPidUuid

payload = ""
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)
dataPlaneAddress = response.json()["dataPlaneAddress"]

pprint.pprint(response.json())

{'dataPlaneAddress': 'http://ds-consumer:1235/0c0e6597-504c-4e33-b0f4-09af4d45fdd9/data/pull/be870ca3-17be-468d-9cf4-5bac748af80d'}


## Use data address to get data from data space

In [93]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")
df = pd.read_json(url + "/comments")
df

,postId,id,name,email,body
0,1,1,id labore ex et quam laborum,Eliseo@gardner.biz,laudantium enim quasi est quidem magnam volupt...
1,1,2,quo vero reiciendis velit similique earum,Jayne_Kuhic@sydney.com,est natus enim nihil est dolore omnis voluptat...
2,1,3,odio adipisci rerum aut animi,Nikita@garfield.biz,quia molestiae reprehenderit quasi aspernatur\...
3,1,4,alias odio sit,Lew@alysha.tv,non et atque\noccaecati deserunt quas accusant...
4,1,5,vero eaque aliquid doloribus et culpa,Hayden@althea.biz,harum non quasi et ratione\ntempore iure ex vo...
...,...,...,...,...,...
495,100,496,et occaecati asperiores quas voluptas ipsam no...,Zola@lizzie.com,neque unde voluptatem iure\nodio excepturi ips...
496,100,497,doloribus dolores ut dolores occaecati,Dolly@mandy.co.uk,non dolor consequatur\nlaboriosam ut deserunt ...
497,100,498,dolores minus aut libero,Davion@eldora.net,aliquam pariatur suscipit fugiat eos sunt\nopt...
498,100,499,excepturi sunt cum a et rerum quo voluptatibus...,Wilburn_Labadie@araceli.name,et necessitatibus tempora ipsum quaerat invent...


## Suspend transfer

In [94]:
url = data_space_consumer + "/api/v1/suspend-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:be870ca3-17be-468d-9cf4-5bac748af80d',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:be870ca3-17be-468d-9cf4-5bac748af80d',
                     'dspace:providerPid': 'urn:uuid:69bf11a8-96f5-4539-9df7-fc11ab0f8612',
                     'dspace:state': 'dspace:SUSPENDED'}}


## Test data access has been suspended

**Disclaimer**: Data shouldn't appear. But for this deployment does appear because an internal refactor we're facing.
In a couple of days this functionality will work like is expected.

In [96]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")
df = pd.read_json(url + "/comments")
df

,postId,id,name,email,body
0,1,1,id labore ex et quam laborum,Eliseo@gardner.biz,laudantium enim quasi est quidem magnam volupt...
1,1,2,quo vero reiciendis velit similique earum,Jayne_Kuhic@sydney.com,est natus enim nihil est dolore omnis voluptat...
2,1,3,odio adipisci rerum aut animi,Nikita@garfield.biz,quia molestiae reprehenderit quasi aspernatur\...
3,1,4,alias odio sit,Lew@alysha.tv,non et atque\noccaecati deserunt quas accusant...
4,1,5,vero eaque aliquid doloribus et culpa,Hayden@althea.biz,harum non quasi et ratione\ntempore iure ex vo...
...,...,...,...,...,...
495,100,496,et occaecati asperiores quas voluptas ipsam no...,Zola@lizzie.com,neque unde voluptatem iure\nodio excepturi ips...
496,100,497,doloribus dolores ut dolores occaecati,Dolly@mandy.co.uk,non dolor consequatur\nlaboriosam ut deserunt ...
497,100,498,dolores minus aut libero,Davion@eldora.net,aliquam pariatur suscipit fugiat eos sunt\nopt...
498,100,499,excepturi sunt cum a et rerum quo voluptatibus...,Wilburn_Labadie@araceli.name,et necessitatibus tempora ipsum quaerat invent...


## Restart transfer

In [13]:
url = data_space_consumer + "/api/v1/restart-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:d14e3346-0072-4c2e-ada7-1f343bcbacf4',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:d14e3346-0072-4c2e-ada7-1f343bcbacf4',
                     'dspace:providerPid': 'urn:uuid:57473891-2fc8-4dd5-949b-20e5109d6e7e',
                     'dspace:state': 'dspace:STARTED'}}


## Test data is accessible again

In [97]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")
df = pd.read_json(url + "/comments")
df

,postId,id,name,email,body
0,1,1,id labore ex et quam laborum,Eliseo@gardner.biz,laudantium enim quasi est quidem magnam volupt...
1,1,2,quo vero reiciendis velit similique earum,Jayne_Kuhic@sydney.com,est natus enim nihil est dolore omnis voluptat...
2,1,3,odio adipisci rerum aut animi,Nikita@garfield.biz,quia molestiae reprehenderit quasi aspernatur\...
3,1,4,alias odio sit,Lew@alysha.tv,non et atque\noccaecati deserunt quas accusant...
4,1,5,vero eaque aliquid doloribus et culpa,Hayden@althea.biz,harum non quasi et ratione\ntempore iure ex vo...
...,...,...,...,...,...
495,100,496,et occaecati asperiores quas voluptas ipsam no...,Zola@lizzie.com,neque unde voluptatem iure\nodio excepturi ips...
496,100,497,doloribus dolores ut dolores occaecati,Dolly@mandy.co.uk,non dolor consequatur\nlaboriosam ut deserunt ...
497,100,498,dolores minus aut libero,Davion@eldora.net,aliquam pariatur suscipit fugiat eos sunt\nopt...
498,100,499,excepturi sunt cum a et rerum quo voluptatibus...,Wilburn_Labadie@araceli.name,et necessitatibus tempora ipsum quaerat invent...


## Complete transfer

In [15]:
url = data_space_consumer + "/api/v1/complete-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:d14e3346-0072-4c2e-ada7-1f343bcbacf4',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:d14e3346-0072-4c2e-ada7-1f343bcbacf4',
                     'dspace:providerPid': 'urn:uuid:57473891-2fc8-4dd5-949b-20e5109d6e7e',
                     'dspace:state': 'dspace:COMPLETED'}}


## Data is not accessible

In [98]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")
df = pd.read_json(url + "/comments")
df

,postId,id,name,email,body
0,1,1,id labore ex et quam laborum,Eliseo@gardner.biz,laudantium enim quasi est quidem magnam volupt...
1,1,2,quo vero reiciendis velit similique earum,Jayne_Kuhic@sydney.com,est natus enim nihil est dolore omnis voluptat...
2,1,3,odio adipisci rerum aut animi,Nikita@garfield.biz,quia molestiae reprehenderit quasi aspernatur\...
3,1,4,alias odio sit,Lew@alysha.tv,non et atque\noccaecati deserunt quas accusant...
4,1,5,vero eaque aliquid doloribus et culpa,Hayden@althea.biz,harum non quasi et ratione\ntempore iure ex vo...
...,...,...,...,...,...
495,100,496,et occaecati asperiores quas voluptas ipsam no...,Zola@lizzie.com,neque unde voluptatem iure\nodio excepturi ips...
496,100,497,doloribus dolores ut dolores occaecati,Dolly@mandy.co.uk,non dolor consequatur\nlaboriosam ut deserunt ...
497,100,498,dolores minus aut libero,Davion@eldora.net,aliquam pariatur suscipit fugiat eos sunt\nopt...
498,100,499,excepturi sunt cum a et rerum quo voluptatibus...,Wilburn_Labadie@araceli.name,et necessitatibus tempora ipsum quaerat invent...


## Protocol state cannot change

In [17]:
url = data_space_consumer + "/api/v1/restart-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:d14e3346-0072-4c2e-ada7-1f343bcbacf4',
 'error': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
           '@type': 'dspace:TransferError',
           'dspace:code': 'TRANSFER_ERROR_CODE',
           'dspace:consumerPid': '123',
           'dspace:providerPid': '123',
           'dspace:reason': ['Protocol Error. dspace:TransferRequestMessage is '
                             'not allowed here. Current state is '
                             'dspace:COMPLETED']}}
